<h2><center>RFM Customer Segmentation in Python and Spark</center></h2>

<h3>RFM segmentation is a great method to divide customers into equal groups depending on three criteria (which compose the RFM acronym):</h3>

Recency. How much time has elapsed since a customer’s last activity or transaction with the company?

Frequency. How often has a customer transacted with the company during a particular period of time?

Monetary. How much a customer has spent with the brand during a particular period of time.

<h3>Setting Up the Environment</h3>

In [1]:
import findspark
findspark.init()

import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
# create an entry point of a spark application
spark = SparkSession \
    .builder \
    .appName('RFM Analysis with PySpark') \
    .getOrCreate()

In [3]:
spark

<h3>Preprocessing Data</h3>

In [4]:
# load data
data = spark.read.format("csv").option("header", "true").load("data.csv")

In [5]:
# inspect data types
data.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)



In [6]:
# define column datatypes for spark
data = data.withColumn("Quantity", data["Quantity"].cast(IntegerType()))
data = data.withColumn("UnitPrice", data["UnitPrice"].cast(DoubleType()))
data = data.withColumn("Date", to_date(unix_timestamp("InvoiceDate", "MM/dd/yyyy").cast("timestamp")))

# define Total column
data = data.withColumn("Total", round(data["UnitPrice"] * data["Quantity"], 2))

# calculate difference in days between 2011-12-31 and the Invoice Date
data = data.withColumn("RecencyDays", expr("datediff('2011-12-31', Date)"))

In [7]:
data

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: string, Country: string, Date: date, Total: double, RecencyDays: int]

<h3>Create of RFM Table</h3>

For each customer, we need to measure the following indicators:

Recency. We shall consider as recency the minimum RecencyDays that we calculated in the previous step. This makes sense since the minimum RecencyDays will give us the number of days that have passed since the customer’s last purchase.

Frequency. It is calculated as the number of purchases that the customer has made.

Monetary. It is the total value of all purchases for each customer.

In [8]:
# create the table
rfm_table = data.groupBy("CustomerId")\
                        .agg(min("RecencyDays").alias("Recency"), \
                             count("InvoiceNo").alias("Frequency"), \
                             sum("Total").alias("Monetary"))

In [9]:
# show the top 5 rows
rfm_table.show()

+----------+-------+---------+------------------+
|CustomerId|Recency|Frequency|          Monetary|
+----------+-------+---------+------------------+
|     16250|    283|       24|            389.44|
|     15574|    199|      168| 702.2500000000002|
|     15555|     34|      925| 4758.200000000001|
|     15271|     29|      275|2485.8199999999997|
|     17714|    342|       10|             153.0|
|     17686|     29|      286|           5739.46|
|     13865|     80|       30|501.55999999999995|
|     14157|     41|       49|400.43000000000006|
|     13610|     34|      228|           1115.43|
|     13772|     55|      177|1132.1299999999999|
|     13282|     40|       40|           1047.84|
|     12394|     85|       27|           1272.48|
|     16320|    194|       56|1038.4600000000003|
|     13192|    117|       63|            911.94|
|     14887|    101|        6|            1862.0|
|     17506|     92|       16|294.28999999999996|
|     17427|     93|        2|             100.8|


In [10]:
# computer RFM indicator, assigning a score ranging from 1 to 4 to each customer with statistical quartiles.
# The customer who has 4s at all indicators is considered the best customer.

r_quartile = rfm_table.approxQuantile("Recency", [0.25, 0.5, 0.75], 0)
f_quartile = rfm_table.approxQuantile("Frequency", [0.25, 0.5, 0.75], 0)
m_quartile = rfm_table.approxQuantile("Monetary", [0.25, 0.5, 0.75], 0)

rfm_table = rfm_table.withColumn("R_Quartile", \
                                 when(col("Recency") >= r_quartile[2] , 1).\
                                 when(col("Recency") >= r_quartile[1] , 2).\
                                 when(col("Recency") >= r_quartile[0] , 3).\
                                 otherwise(4))

rfm_table = rfm_table.withColumn("F_Quartile", \
                                 when(col("Frequency") > f_quartile[2] , 4).\
                                 when(col("Frequency") > f_quartile[1] , 3).\
                                 when(col("Frequency") > f_quartile[0] , 2).\
                                 otherwise(1))

rfm_table = rfm_table.withColumn("M_Quartile", \
                                 when(col("Monetary") >= m_quartile[2] , 4).\
                                 when(col("Monetary") >= m_quartile[1] , 3).\
                                 when(col("Monetary") >= m_quartile[0] , 2).\
                                 otherwise(1))

rfm_table = rfm_table.withColumn("RFM_Score", concat(col("R_Quartile"), col("F_Quartile"), col("M_Quartile")))

In [11]:
# the final version of RFM table
rfm_table.show(10)

+----------+-------+---------+------------------+----------+----------+----------+---------+
|CustomerId|Recency|Frequency|          Monetary|R_Quartile|F_Quartile|M_Quartile|RFM_Score|
+----------+-------+---------+------------------+----------+----------+----------+---------+
|     16250|    283|       24|            389.44|         1|         2|         2|      122|
|     15574|    199|      168| 702.2500000000002|         1|         4|         3|      143|
|     15555|     34|      925| 4758.200000000001|         4|         4|         4|      444|
|     15271|     29|      275|2485.8199999999997|         4|         4|         4|      444|
|     17714|    342|       10|             153.0|         1|         1|         1|      111|
|     17686|     29|      286|           5739.46|         4|         4|         4|      444|
|     13865|     80|       30|501.55999999999995|         2|         2|         2|      222|
|     14157|     41|       49|400.43000000000006|         3|         3

<h3>RFM Analysis</h3>

In [12]:
# find the best customers
rfm_table.select('CustomerID').where('RFM_Score == 444').show(10)

+----------+
|CustomerID|
+----------+
|     15555|
|     15271|
|     17686|
|     17757|
|     16549|
|     13985|
|     14525|
|     18283|
|     12957|
|     17491|
+----------+
only showing top 10 rows



In [18]:
pip install plotly==4.14.3

  Stored in directory: C:\Users\titi1\AppData\Local\pip\Cache\wheels\d7\a9\33\acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying
Note: you may need to restart the kernel to use updated packages.


In [21]:
import plotly.graph_objects as go
from plotly.offline import iplot

In [22]:
# segment customers based on the RFM score, count the customers in each segment
# convert the Spark dataframe to pandas dataframe

grouped_by_rfmscore = rfm_table.groupBy("RFM_Score").count().orderBy("count", ascending=False)
grouped_by_rfmscore_pandas = grouped_by_rfmscore.toPandas()

data = [go.Bar(x=grouped_by_rfmscore_pandas['RFM_Score'], y=grouped_by_rfmscore_pandas['count'])]

layout = go.Layout(
    title=go.layout.Title(
        text='Customer RFM Segments'
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='RFM Segment'
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='Number of Customers'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='rfm_Segments')

The best customers compose the largest segment, but the worst customers (customers with all scores of 1s) are the second-largest segment

<h3>Apart from the best and worst customers, we may also define the following important customer segments:</h3>

<h4>High-spending New Customers</h4> This group consists of those customers in 4–1–3 and 4–1–4. These are customers who transacted only once, but very recently and they spent a lot.

<h4>Lowest-Spending Active Loyal Customers</h4> This group consists of those customers in segments 3–4–1 and 4–4–1 (they transacted recently and do so often, but spend the least).

<h4>Churned Best Customers</h4> This segment consists of those customers in groups 1–3–3, 1–4–3, 1–3–4 and 1–4–4 (they transacted frequently and spent a lot, but it’s been a long time since they’ve transacted).

Deciding which groups of customers to target and how to best communicate with them is where the art of marketing comes in.